# Test

In [2]:

import os
import sys
import numpy as np
import pandas as pd
import random

from argparse import ArgumentParser
import datetime

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.backends.cudnn as cudnn

from pytvision.transforms import transforms as mtrans
from pytvision import visualization as view

sys.path.append('../')
from torchlib.datasets  import factory
from torchlib.datasets  import gdata
from torchlib.classifierneuralnet import NeuralNetClassifier
from torchlib.datasets.imaterialist import IMaterialistImageDataset

from misc import get_transforms_aug, get_transforms_det, get_transforms_hflip, get_transforms_gray, get_transforms_aug2
from sklearn import metrics

In [3]:
home = os.getenv("HOME")
pathname= os.path.join(home, '.datasets')
name_dataset='imaterialist'
project='../out/netruns'
no_cuda=False
seed=1
gpu=0
name='exp_baseline_inception_v3_cross_adam_imaterialist_002'
model='chk000039.pth.tar'
path_model=os.path.join(project,name,'models',model)
batch_size=80
workers=80
num_input_channels=3
pathdata=os.path.join( pathname, name_dataset, 'test' )

pathnameout = os.path.join(project, 'predicts')


print(pathname)
print(project)
print(name_dataset)
print(path_model)
print(pathdata)


/home/fernandez/.datasets
../out/netruns
imaterialist
../out/netruns/exp_baseline_inception_v3_cross_adam_imaterialist_002/models/chk000039.pth.tar
/home/fernandez/.datasets/imaterialist/test


In [4]:

# neuralnet
network = NeuralNetClassifier(
    patchproject=project,
    nameproject=name,
    no_cuda=no_cuda,
    seed=seed,
    gpu=gpu
    )

cudnn.benchmark = True

# load model
if network.load( path_model ) is not True:
    assert(False)

tta_preprocess = [ 
    get_transforms_det(network.size_input), 
    get_transforms_hflip(network.size_input), 
    #get_transforms_gray(network.size_input),
    #get_transforms_aug2(network.size_input),
    #get_transforms_aug2(network.size_input),    
    ]

dataloaders = []
for transform in tta_preprocess:    
    # test dataset
    data = IMaterialistImageDataset(
        pathname=pathdata,
        ext='jpg',
        num_channels=num_input_channels,
        transform=transform
        )
    dataloader = DataLoader(data, batch_size=batch_size, shuffle=False, num_workers=workers )
    dataloaders.append(dataloader)

data_test = data
print(len(dataloaders))
    

=> loading checkpoint '../out/netruns/exp_baseline_inception_v3_cross_adam_imaterialist_002/models/chk000039.pth.tar'
=> loaded checkpoint for inception_v3 arch!
2


In [5]:

random.seed( seed )
files = [ f for f in sorted(os.listdir(pathnameout)) if f.split('.')[-1] == 'csv' ]; 

for i,data in enumerate(dataloaders):
    Ids, Yhat = network.predict( data )
    df = pd.DataFrame( np.concatenate((Ids, Yhat), axis=1) )
    df.to_csv( os.path.join(pathnameout , 'test_{}_dp{}.csv'.format(name, i + len(files) ) ), index=False, encoding='utf-8')        

print('dir: {}'.format(pathnameout))
print('DONE!!!')


100%|██████████| 159/159 [00:40<00:00,  3.92it/s]


dir: ../out/netruns/predicts
DONE!!!


 # Multiclasification systems
 
       +-----+  dp_1
     --| cnn |---------+
       +-----+         |
                       | 
       +-----+  dp_2   |     +--------+
     --| cnn |---------+ ----| Fusion |-----------+
       +-----+         |     +--------+
                       .        |
                       .        SOFT, HARD, TR
                       .        Soft: sRp, sRs, sRmx, sRmi, sRmd, sRmv
       +-----+  dp_L   |        Hard: hWmv, hRec, hNb  
     --| cnn |---------+        Train: tFi, tLop, tMb, tMsvm
       +-----+


In [6]:

## On Combining Classifiers
# - Combining soft ruler
# - https://dspace.cvut.cz/bitstream/handle/10467/9443/1998-On-combining-classifiers.pdf?sequence=1
    
def product_ruler( dp, P=None ):
    """
    Ecuation. Josef Kittler [7]
    https://dspace.cvut.cz/bitstream/handle/10467/9443/1998-On-combining-classifiers.pdf?sequence=1
    Soft Product Rule    
    P^{(-(R-1))}( w_j )\prod_iP(w_j|x_i)  = \max_k P^{(-(R-1))}( w_k ) \prod_i
    P(w_k|x_i) (1)
    which under the assumption of equal priors, simplifies to the following:
    \prod_iP(w_j|x_i)  = \max_k \prod_i P(w_k|x_i) (2)    
    Args:
        @dp: []_nxcxl
        @P: class prior []_c
    """    
    p = dp.prod(2)
    if P is not None:
        l = dp.shape[2]
        p = P**-(l-1)*p
    return p

def sum_ruler( dp, P=None ):
    """
    Ecuation. Josef Kittler [11]
    https://dspace.cvut.cz/bitstream/handle/10467/9443/1998-On-combining-classifiers.pdf?sequence=1
    Soft Sum Ruler
    $$(1-R)P( w_j ) + \sum_iP(w_j|x_i)  = \max_k [ (1-R)P(w_k) + \sum_iP(w_k|x_i)]$$
    which under the assumption of equal priors simplifies to the following:
    $$\sum_iP(w_j|x_i)  = \max_k \sum_iP(w_k|x_i)$$    
    Args:
        @dp: []_nxcxl
    """
    p = dp.sum(2)
    if P is not None:
        l = dp.shape[2]
        p = (1-l)*P + p
    return p

def max_ruler( dp, P=None ):
    """
    Ecuation. Josef Kittler [14][15]
    https://dspace.cvut.cz/bitstream/handle/10467/9443/1998-On-combining-classifiers.pdf?sequence=1
    Soft Max Ruler
    Args:
        @dp: []_nxcxl
    """
    p = dp.max(2)
    if P is not None:
        l = dp.shape[2]
        p = (1-l)*P + l*p
    return p

def min_ruler( dp, P=None ):
    """
    Ecuation. Josef Kittler [16]
    https://dspace.cvut.cz/bitstream/handle/10467/9443/1998-On-combining-classifiers.pdf?sequence=1
    Soft Min Ruler
    Args:
        @dp: []_nxcxl
    """
    p = dp.min(2)
    if P is not None:
        l = dp.shape[2]
        p = P**-(l-1)*p
        p = (1-l)*P + l*p
    return p


def majority_ruler( dp  ):
    """
    Ecuation. Josef Kittler [20]
    https://dspace.cvut.cz/bitstream/handle/10467/9443/1998-On-combining-classifiers.pdf?sequence=1
    Soft Majority Vote Rule
    Args:
        @dp: []_nxcxl
    """
    n,c,l = dp.shape
    p = np.argmax(dp,axis=1)
    
    dki = np.zeros((n,c))
    for i in range(n):
        tup = p[i,:]
        for j in range(c):
            dki[i,j] = np.sum( tup == j )
        
    p=dki
    return p


def mean_ruler( dp ):
    """
    Ecuation. Josef Kittler [18]
    https://dspace.cvut.cz/bitstream/handle/10467/9443/1998-On-combining-classifiers.pdf?sequence=1
    Soft Median Ruler
    Args:
        @dp: []_nxcxl
    """
    p = dp.mean(2)
    return p
    

    
# # test
# #[n,c,l]
# #dp = np.random.rand(10,4,3 )
# #P = np.array([0.1,0.1,0.1,0.7])
# dp = np.array( [[[0.2,0,0,0],[0.8,1.0,1.0,1.0]],[[0.3,0.6,0.9,0.5],[0.7,0.4,0.1,0.5]]] )
# P = np.array([0.7,0.3])
# print(dp.shape)
# #print(dp[:,:,0])

# func = [product_ruler, sum_ruler, max_ruler, min_ruler, majority_ruler, mean_ruler]
# for f in func:
#     p = f(dp)
#     print( p.argmax(1) )
    
    

In [7]:
files = [ f for f in sorted(os.listdir(pathnameout)) if f.split('.')[-1] == 'csv' ]; 
print(files)

l = len(files)
dp =[]; ids=[]
for f in files:  
    mdata = pd.read_csv( os.path.join(pathnameout , f )  )
    dpdata = mdata.as_matrix()
    ids.append(dpdata[:,0])    
    dp.append( dpdata[:,1:] )
    
dp = np.array(dp).transpose((1,2,0))
ids = np.array(ids)

assert( not (ids[0,:]-ids[:1,:]).sum() )
print(dp.shape)
    

['test_exp_baseline_inception_v3_cross_adam_imaterialist_002_dp0.csv', 'test_exp_baseline_inception_v3_cross_adam_imaterialist_002_dp1.csv']
(12704, 128, 2)


In [8]:
Id = ids[0,:]
p = mean_ruler(dp)
pred = np.argmax(p, axis=1)

print(dp.shape)
print(p.shape)
print(pred.shape)

(12704, 128, 2)
(12704, 128)
(12704,)


In [9]:
pred[:100]

array([ 52, 125,  52,  76,  52, 127,  71,  23,  23,  52,  76,  52,  67,
        76,   4,   8, 125,  79,  76,  26,  59,  76, 125,  95,  76,  23,
        76,   3,  47,  57,  76,  40,  76, 103,  78, 106,  76,  49,  52,
        43,  76,  13,  52,  71,  76,  59,  76,  52,  52,  76,  76,  76,
        59,  76, 103,  76,  76,  79,  87, 125,  52,  76,  76,  80,  95,
        95,  76, 125,  76,  76,  93,  52,  43, 125,  59,  76,  23, 119,
       121,  76,  76,  76,  76,  64,  95,  18,  76,  23,  76,  76,  23,
        76,  23,  38,  52,  68,  59,  76,  76,  76])

# Submission

In [10]:
data_val=factory.FactoryDataset.factory(
            pathname=pathname, 
            name=name_dataset, 
            subset=factory.validation, 
            download=True )

print(len(data_val))
print(data_val.class_to_idx)
print(data_val.classes)

6309
{'1': 0, '10': 1, '100': 2, '101': 3, '102': 4, '103': 5, '104': 6, '105': 7, '106': 8, '107': 9, '108': 10, '109': 11, '11': 12, '110': 13, '111': 14, '112': 15, '113': 16, '114': 17, '115': 18, '116': 19, '117': 20, '118': 21, '119': 22, '12': 23, '120': 24, '121': 25, '122': 26, '123': 27, '124': 28, '125': 29, '126': 30, '127': 31, '128': 32, '13': 33, '14': 34, '15': 35, '16': 36, '17': 37, '18': 38, '19': 39, '2': 40, '20': 41, '21': 42, '22': 43, '23': 44, '24': 45, '25': 46, '26': 47, '27': 48, '28': 49, '29': 50, '3': 51, '30': 52, '31': 53, '32': 54, '33': 55, '34': 56, '35': 57, '36': 58, '37': 59, '38': 60, '39': 61, '4': 62, '40': 63, '41': 64, '42': 65, '43': 66, '44': 67, '45': 68, '46': 69, '47': 70, '48': 71, '49': 72, '5': 73, '50': 74, '51': 75, '52': 76, '53': 77, '54': 78, '55': 79, '56': 80, '57': 81, '58': 82, '59': 83, '6': 84, '60': 85, '61': 86, '62': 87, '63': 88, '64': 89, '65': 90, '66': 91, '67': 92, '68': 93, '69': 94, '7': 95, '70': 96, '71': 97, '7

In [11]:

submission_filepath = '../submission.csv'
submission = pd.read_csv('~/.kaggle/competitions/imaterialist-challenge-furniture-2018/sample_submission_randomlabel.csv')

TIds = np.array([ int(data_test.getId( int(i) )) for i in Id ])
TPred = np.array( [ int(data_val.classes[c]) for c in pred  ] )
submission.loc[TIds-1, 'predicted'] = TPred
submission.head()



,id,predicted
0,1,30
1,2,52
2,3,91
3,4,54
4,5,126


In [12]:
submission.to_csv(submission_filepath, index=False, encoding='utf-8')
print('SAVE!!!')

SAVE!!!
